In [36]:
thresholds_df.to_excel("data.xlsx", index=False)

In [1]:
from common_functions import get_secret, ret_metabase, google_sheets, dwh_query
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [35]:
data = dwh_query("Egypt", """SELECT
  t.created_at,
  t.date,
  t.retailer_id,
  t.id,
  t.visit_id,
  tv.agent_id,
  workflow_id
    FROM
  main_egypt_logistics.tasks t
  LEFT JOIN main_egypt_logistics.task_visits tv ON tv.id = t.visit_id
  LEFT JOIN task_statuses ts ON ts.id = t.status_id
  LEFT JOIN main_egypt_logistics.agents a ON a.id = tv.agent_id
  LEFT JOIN main_egypt_logistics.PARTNER_SUPERVISORS ass ON ass.agent_id = a.PARTNER_INTEGRATION_ID
 
    WHERE
  t.status_id = 1
  AND t.date = CURRENT_DATE
  """, "read", columns=[], conn=None)
data

/home/ec2-user/service_account_key.json


,created_at,date,retailer_id,id,visit_id,agent_id,workflow_id
0,2025-09-16 15:52:46.212756,2025-09-16,7767,6607124,None,None,7
1,2025-09-16 15:52:39.133945,2025-09-16,56453,6606580,None,None,5
2,2025-09-16 15:52:39.133945,2025-09-16,56453,6606581,None,None,2
3,2025-09-16 15:52:39.133945,2025-09-16,56453,6606582,None,None,7
4,2025-09-16 15:52:39.133945,2025-09-16,65782,6606583,None,None,7
...,...,...,...,...,...,...,...
1719,2025-09-16 15:53:04.210791,2025-09-16,569716,6608301,None,None,7
1720,2025-09-16 15:53:04.210791,2025-09-16,577779,6608302,None,None,7
1721,2025-09-16 15:53:04.210791,2025-09-16,577779,6608303,None,None,5
1722,2025-09-16 15:52:54.265543,2025-09-16,274431,6607686,None,None,7


In [2]:
df = ret_metabase("Egypt", 62832)
df

/home/ec2-user/service_account_key.json


,Task ID


In [10]:
google_sheets("On_Ground_Dispatch_LOGS", "fetched_data", "overwrite", df=creation_df)

/home/ec2-user/service_account_key.json


'data is added to the sheet successfully'

# Manual dispatching

In [ ]:
from common_functions import get_secret
from datetime import datetime, timedelta
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO

import requests
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token

#------------------------
# Load Data from Local File
#------------------------

# Specify the file path - you can change this to your actual file name
INPUT_FILE_PATH = "Dispatch batch 2 .xlsx"  # Change this to your file name

try:
    # Load data from local file
    df = pd.read_excel(INPUT_FILE_PATH)
    print(f"✓ Successfully loaded data from {INPUT_FILE_PATH}")
    print(f"Total rows loaded: {len(df)}")
    print(f"Columns in file: {list(df.columns)}")
    
    # Prepare creation data - adjust column names as needed
    creation_df = df.copy()
    # Make sure these column names match your Excel file
    required_columns = ['Retailer ID', 'Workflow ID', 'Date', 'Description']
    
    # Check if required columns exist
    missing_columns = [col for col in required_columns if col not in creation_df.columns]
    if missing_columns:
        print(f"⚠️  Warning: Missing columns in your file: {missing_columns}")
        print(f"Available columns: {list(creation_df.columns)}")
        print("Please make sure your file has the required columns or update the column names in the code.")
    
    # Select only the required columns (if they exist)
    available_columns = [col for col in required_columns if col in creation_df.columns]
    creation_df = creation_df[available_columns]
    
except FileNotFoundError:
    print(f"✗ Error: File '{INPUT_FILE_PATH}' not found in the current directory.")
    print("Please make sure your Excel file is in the same folder as this script.")
    print("You can change the INPUT_FILE_PATH variable to match your file name.")
    exit()
except Exception as e:
    print(f"✗ Error loading file: {str(e)}")
    exit()

AT = get_jwt()

#------------------------
# Bulk create Task with Batch Processing
#------------------------

def process_batch(batch_df, batch_number, total_batches):
    """
    Process a single batch of data for bulk task creation
    """
    # Create a temporary file for this batch
    batch_file_path = f"manual_dispatch_batch_{batch_number}.xlsx"
    
    try:
        # Save batch DataFrame to an Excel file
        batch_df.to_excel(batch_file_path, index=False)
        
        # Prepare the request
        url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/creation"
        
        headers = {
            "accept": "application/json, text/plain, */*",
            "authorization": f"Bearer {AT}",
            "country_identifier": "EG",
            "language": "AR",
            "origin": "https://logistics.maxab.info",
            "referer": "https://logistics.maxab.info/",
            "user-agent": "Mozilla/5.0",
        }
        
        files = {
            "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
        }
        
        # Send POST request with file upload
        response = requests.post(url, headers=headers, files=files)
        
        print(f"Batch {batch_number}/{total_batches} - Response: {response.status_code}")
        
        if response.status_code in [200, 201]:
            print(f"✓ Batch {batch_number} processed successfully")
        else:
            print(f"✗ Batch {batch_number} failed with status {response.status_code}")
            print(f"Response: {response.text}")
            
        return response.status_code in [200, 201]
        
    except Exception as e:
        print(f"✗ Error processing batch {batch_number}: {str(e)}")
        return False
        
    finally:
        # Clean up temporary file
        if os.path.exists(batch_file_path):
            os.remove(batch_file_path)

# Calculate batch size and number of batches
BATCH_SIZE = 500
total_rows = len(creation_df)
total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division

print(f"\n=== Manual Dispatch - Bulk Create Tasks ===")
print(f"Total rows to process: {total_rows}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Total batches: {total_batches}")

# Process data in batches
successful_batches = 0
failed_batches = 0

for batch_num in range(1, total_batches + 1):
    start_idx = (batch_num - 1) * BATCH_SIZE
    end_idx = min(batch_num * BATCH_SIZE, total_rows)
    
    batch_df = creation_df.iloc[start_idx:end_idx].copy()
    
    print(f"\nProcessing batch {batch_num}/{total_batches} (rows {start_idx + 1}-{end_idx})")
    
    success = process_batch(batch_df, batch_num, total_batches)
    
    if success:
        successful_batches += 1
    else:
        failed_batches += 1

print(f"\n=== Manual Dispatch - Batch Processing Complete ===")
print(f"Successful batches: {successful_batches}/{total_batches}")
print(f"Failed batches: {failed_batches}/{total_batches}")
print(f"Success rate: {(successful_batches/total_batches)*100:.1f}%")

#--------------------------
# Export Data
#--------------------------

import requests

url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/export"

# Get current date in YYYY-MM-DD format
current_date = datetime.now().strftime("%Y-%m-%d")
querystring = {"date_from": current_date, "date_to": current_date, "status_ids": "1"}

payload = ""
headers = {"Authorization": f"Bearer {AT}"}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

print(f"\n2. Export Data Response: {response.status_code}")

response = requests.get(url, headers=headers, params=querystring)

#--------------------------
# Merge DataFrames
#--------------------------

# Load Excel directly into pandas
export_df = pd.read_excel(BytesIO(response.content))
export_df = export_df[export_df['Status'] == 'CREATED']

import pandas as pd

# Merge on common columns
merged_df = export_df.merge(
    df[['Retailer ID', 'AGENT_ID']],   # only keep necessary columns
    how='left',                        # keep all rows from export_df
    left_on='Retailer Id',             # column name in export_df
    right_on='Retailer ID'             # column name in df
)

# Drop duplicate 'Retailer ID' (optional, since export_df already has 'Retailer Id')
merged_df = merged_df.drop(columns=['Retailer ID', 'Task Id'])

merged_df = merged_df[['Task Id', 'AGENT_ID']]

#--------------------------
# Bulk Dispatch Tasks with Batch Processing
#--------------------------

def process_dispatch_batch(batch_df, batch_number, total_batches):
    """
    Process a single batch of data for bulk task dispatching
    """
    # Create a temporary file for this batch
    batch_file_path = f"manual_dispatch_BD_batch_{batch_number}.xlsx"
    
    try:
        # Save batch DataFrame to an Excel file
        batch_df.to_excel(batch_file_path, index=False)
        
        # Prepare the request
        url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/dispatching"
        
        headers = {
            "accept": "application/json, text/plain, */*",
            "authorization": f"Bearer {AT}",
            "country_identifier": "EG",
            "language": "AR",
            "origin": "https://logistics.maxab.info",
            "referer": "https://logistics.maxab.info/",
            "user-agent": "Mozilla/5.0",
        }
        
        files = {
            "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
        }
        
        # Send POST request with file upload
        response = requests.post(url, headers=headers, files=files)
        
        print(f"Dispatch Batch {batch_number}/{total_batches} - Response: {response.status_code}")
        
        if response.status_code in [200, 201]:
            print(f"✓ Dispatch Batch {batch_number} processed successfully")
        else:
            print(f"✗ Dispatch Batch {batch_number} failed with status {response.status_code}")
            print(f"Response: {response.text}")
            
        return response.status_code in [200, 201]
        
    except Exception as e:
        print(f"✗ Error processing dispatch batch {batch_number}: {str(e)}")
        return False
        
    finally:
        # Clean up temporary file
        if os.path.exists(batch_file_path):
            os.remove(batch_file_path)

# Calculate batch size and number of batches for dispatch
DISPATCH_BATCH_SIZE = 500
total_dispatch_rows = len(merged_df)
total_dispatch_batches = (total_dispatch_rows + DISPATCH_BATCH_SIZE - 1) // DISPATCH_BATCH_SIZE  # Ceiling division

print(f"\n=== Manual Dispatch - Bulk Dispatch Tasks ===")
print(f"Total dispatch rows to process: {total_dispatch_rows}")
print(f"Dispatch batch size: {DISPATCH_BATCH_SIZE}")
print(f"Total dispatch batches: {total_dispatch_batches}")

# Process dispatch data in batches
successful_dispatch_batches = 0
failed_dispatch_batches = 0

for batch_num in range(1, total_dispatch_batches + 1):
    start_idx = (batch_num - 1) * DISPATCH_BATCH_SIZE
    end_idx = min(batch_num * DISPATCH_BATCH_SIZE, total_dispatch_rows)
    
    batch_df = merged_df.iloc[start_idx:end_idx].copy()
    
    print(f"\nProcessing dispatch batch {batch_num}/{total_dispatch_batches} (rows {start_idx + 1}-{end_idx})")
    
    success = process_dispatch_batch(batch_df, batch_num, total_dispatch_batches)
    
    if success:
        successful_dispatch_batches += 1
    else:
        failed_dispatch_batches += 1

print(f"\n=== Manual Dispatch - Dispatch Batch Processing Complete ===")
print(f"Successful dispatch batches: {successful_dispatch_batches}/{total_dispatch_batches}")
print(f"Failed dispatch batches: {failed_dispatch_batches}/{total_dispatch_batches}")
print(f"Dispatch success rate: {(successful_dispatch_batches/total_dispatch_batches)*100:.1f}%")


In [3]:
df = google_sheets("On_Ground_Dispatch_LOGS", "fetched_data", "get")
df

/home/ec2-user/service_account_key.json


,RETAILER_ID,Workflow ID,Date,Description
0,449990,5,2025-09-14,عميل نشط على سوق التجار اخر 60 يوم؛اعمل تالت ا...
1,449990,2,2025-09-14,2500- مبيعات مدفوعات: عميل طلبات مناسب انه يكو...
2,449990,7,2025-09-14,تسجيل بينات العميل
3,451159,7,2025-09-14,تسجيل بينات العميل
4,451159,5,2025-09-14,عميل نشط على سوق التجار اخر 60 يوم؛اعمل اول او...
...,...,...,...,...
10720,651682,6,2025-09-14,تفعيل الآجل، مصاريف إدارية 1.75% في الاسبوع ، ...
10721,651682,1,2025-09-14,عميل نشط - مطلوب اوردر من بضاعة مكسب بقيمة502...
10722,664440,1,2025-09-14,عميل غير نشط اكتر من اسبوعين - مطلوب اوردر من...
10723,672875,6,2025-09-14,تفعيل الآجل، مصاريف إدارية 1.75% في الاسبوع ، ...


In [8]:
from common_functions import get_secret, ret_metabase
from datetime import datetime, timedelta
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO



import requests
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
        
    
    return jwt_access_token
# Separate DataFrame for cancellation with different query ID
df_cancellation = ret_metabase("Egypt", 62832)
cancellation_df = df_cancellation.copy()

AT = get_jwt()

# ------------------------
# Cancellation Row-by-Row (one API call per row)
# ------------------------

def process_cancellation_row(row, row_number, total_rows):
    """
    Process a single row for task cancellation (one API call per row).
    """
    # Create a temporary file for this row
    temp_file_path = f"data_for_cancellation_row_{row_number}.xlsx"

    try:
        # Convert the single row to a one-row DataFrame and save to Excel
        pd.DataFrame([row]).to_excel(temp_file_path, index=False)

        # Prepare the request
        url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/cancellation"

        headers = {
            "accept": "application/json, text/plain, */*",
            "authorization": f"Bearer {AT}",
            "country_identifier": "EG",
            "language": "AR",
            "origin": "https://logistics.maxab.info",
            "referer": "https://logistics.maxab.info/",
            "user-agent": "Mozilla/5.0",
        }

        files = {
            "file": (temp_file_path, open(temp_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
        }

        # Send POST request with file upload
        response = requests.post(url, headers=headers, files=files)

        print(f"Cancellation Row {row_number}/{total_rows} - Response: {response.status_code}")

        if response.status_code in [200, 201]:
            print(f"✓ Cancellation row {row_number} processed successfully")
        else:
            print(f"✗ Cancellation row {row_number} failed with status {response.status_code}")
            print(f"Response: {response.text}")

        return response.status_code in [200, 201]

    except Exception as e:
        print(f"✗ Error processing cancellation row {row_number}: {str(e)}")
        return False

    finally:
        # Clean up temporary file
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

total_cancellation_rows = len(cancellation_df)

print(f"\n=== Row-by-Row Cancellation Tasks ===")
print(f"Total cancellation rows to process: {total_cancellation_rows}")

# Process cancellation data row by row
successful_cancellation_rows = 0
failed_cancellation_rows = 0

for i, (_, row_series) in enumerate(cancellation_df.iterrows(), start=1):
    # Convert Series to dict to avoid pandas index/dtype issues
    row_dict = row_series.to_dict()

    success = process_cancellation_row(row_dict, i, total_cancellation_rows)

    if success:
        successful_cancellation_rows += 1
    else:
        failed_cancellation_rows += 1

print(f"\n=== Cancellation Row Processing Complete ===")
print(f"Successful cancellation rows: {successful_cancellation_rows}/{total_cancellation_rows}")
print(f"Failed cancellation rows: {failed_cancellation_rows}/{total_cancellation_rows}")
print(f"Cancellation success rate: {(successful_cancellation_rows/max(1,total_cancellation_rows))*100:.1f}%")


/home/ec2-user/service_account_key.json

=== Row-by-Row Cancellation Tasks ===
Total cancellation rows to process: 6791
Cancellation Row 1/6791 - Response: 400
✗ Cancellation row 1 failed with status 400
Response: {"code":400,"error_code":"task.is.not.pending","message":"task must be created or dispatched  to be updated","status":false}
Cancellation Row 2/6791 - Response: 400
✗ Cancellation row 2 failed with status 400
Response: {"code":400,"error_code":"task.is.not.pending","message":"task must be created or dispatched  to be updated","status":false}
Cancellation Row 3/6791 - Response: 400
✗ Cancellation row 3 failed with status 400
Response: {"code":400,"error_code":"task.is.not.pending","message":"task must be created or dispatched  to be updated","status":false}
Cancellation Row 4/6791 - Response: 400
✗ Cancellation row 4 failed with status 400
Response: {"code":400,"error_code":"task.is.not.pending","message":"task must be created or dispatched  to be updated","status":false}
Canc

In [26]:
def process_batch(batch_df, batch_number, total_batches):
    """
    Process a single batch of data by creating tasks row-by-row (one API call per row).
    Returns True only if all rows in the batch are created successfully.
    """
    import os
    import requests
    import pandas as pd
    df = ret_metabase("Egypt", 63605)  
    creation_df = df.copy()
    creation_df = creation_df[['RETAILER_ID', 'Workflow ID', 'Date', 'Description']]

    url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/creation"

    headers = {
        "accept": "application/json, text/plain, */*",
        "authorization": f"Bearer {AT}",
        "country_identifier": "EG",
        "language": "AR",
        "origin": "https://logistics.maxab.info",
        "referer": "https://logistics.maxab.info/",
        "user-agent": "Mozilla/5.0",
    }

    total_rows = len(batch_df)
    successful_rows = 0
    failed_rows = 0

    for idx, (_, row) in enumerate(batch_df.iterrows(), start=1):
        temp_file_path = f"data_for_SF_row_b{batch_number}_{idx}.xlsx"
        try:
            # Save single row to Excel
            pd.DataFrame([row]).to_excel(temp_file_path, index=False)

            files = {
                "file": (temp_file_path, open(temp_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
            }

            response = requests.post(url, headers=headers, files=files)

            print(f"Batch {batch_number}/{total_batches} - Row {idx}/{total_rows} - Response: {response.status_code}")

            if response.status_code in [200, 201]:
                successful_rows += 1
            else:
                failed_rows += 1
                print(f"Response: {response.text}")

        except Exception as e:
            failed_rows += 1
            print(f"✗ Error processing batch {batch_number} row {idx}: {str(e)}")
        finally:
            if os.path.exists(temp_file_path):
                os.remove(temp_file_path)

    print(f"✓ Batch {batch_number} completed: {successful_rows}/{total_rows} rows succeeded, {failed_rows} failed")
    return failed_rows == 0

# Calculate batch size and number of batches
BATCH_SIZE = 500
total_rows = len(creation_df)
total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division

print(f"Total rows to process: {total_rows}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Total batches: {total_batches}")

# Process data in batches
successful_batches = 0
failed_batches = 0
total_rows_created = 0  # Track actual rows created

for batch_num in range(1, total_batches + 1):
    start_idx = (batch_num - 1) * BATCH_SIZE
    end_idx = min(batch_num * BATCH_SIZE, total_rows)

    batch_df = creation_df.iloc[start_idx:end_idx].copy()
    batch_row_count = len(batch_df)  # Number of rows in this batch

    print(f"\nProcessing batch {batch_num}/{total_batches} (rows {start_idx + 1}-{end_idx})")

    success = process_batch(batch_df, batch_num, total_batches)

    if success:
        successful_batches += 1
        total_rows_created += batch_row_count  # Add successful batch rows to total
    else:
        failed_batches += 1

print(f"\n=== Batch Processing Complete ===")
print(f"Successful batches: {successful_batches}/{total_batches}")
print(f"Failed batches: {failed_batches}/{total_batches}")
print(f"Success rate: {(successful_batches/total_batches)*100:.1f}%")
print(f"Total rows created: {total_rows_created}")

# Store creation statistics
creation_stats = {
    'total_batches': total_batches,
    'successful_batches': successful_batches,
    'failed_batches': failed_batches,
    'success_rate': (successful_batches/total_batches)*100 if total_batches > 0 else 0,
    'total_rows_created': total_rows_created
}


Total rows to process: 5760
Batch size: 500
Total batches: 12

Processing batch 1/12 (rows 1-500)
/home/ec2-user/service_account_key.json


KeyboardInterrupt: 

In [3]:
from common_functions import get_secret, ret_metabase
from datetime import datetime, timedelta
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO



import requests
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
        
    
    return jwt_access_token


df = ret_metabase("Egypt", 63605)  
creation_df = df.copy()
creation_df = creation_df[['RETAILER_ID', 'Workflow ID', 'Date', 'Description']]

# # Separate DataFrame for cancellation with different query ID
# df_cancellation = ret_metabase("Egypt", 62832)
# cancellation_df = df_cancellation.copy()

AT = get_jwt()

# # ------------------------
# # Bulk Cancellation with Batch Processing (2000 batch limit)
# # ------------------------

# def process_cancellation_batch(batch_df, batch_number, total_batches):
#     """
#     Process a single batch of data for bulk task cancellation
#     """
#     # Create a temporary file for this batch
#     batch_file_path = f"data_for_cancellation_batch_{batch_number}.xlsx"
    
#     try:
#         # Save batch DataFrame to an Excel file
#         batch_df.to_excel(batch_file_path, index=False)
        
#         # Prepare the request
#         url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/cancellation"
        
#         headers = {
#             "accept": "application/json, text/plain, */*",
#             "authorization": f"Bearer {AT}",
#             "country_identifier": "EG",
#             "language": "AR",
#             "origin": "https://logistics.maxab.info",
#             "referer": "https://logistics.maxab.info/",
#             "user-agent": "Mozilla/5.0",
#         }
        
#         files = {
#             "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
#         }
        
#         # Send POST request with file upload
#         response = requests.post(url, headers=headers, files=files)
        
#         print(f"Cancellation Batch {batch_number}/{total_batches} - Response: {response.status_code}")
        
#         if response.status_code in [200, 201]:
#             print(f"✓ Cancellation Batch {batch_number} processed successfully")
#         else:
#             print(f"✗ Cancellation Batch {batch_number} failed with status {response.status_code}")
#             print(f"Response: {response.text}")
            
#         return response.status_code in [200, 201]
        
#     except Exception as e:
#         print(f"✗ Error processing cancellation batch {batch_number}: {str(e)}")
#         return False
        
#     finally:
#         # Clean up temporary file
#         if os.path.exists(batch_file_path):
#             os.remove(batch_file_path)

# # Calculate batch size and number of batches for cancellation
# CANCELLATION_BATCH_SIZE = 2000
# total_cancellation_rows = len(cancellation_df)
# total_cancellation_batches = (total_cancellation_rows + CANCELLATION_BATCH_SIZE - 1) // CANCELLATION_BATCH_SIZE  # Ceiling division

# print(f"\n=== Bulk Cancellation Tasks ===")
# print(f"Total cancellation rows to process: {total_cancellation_rows}")
# print(f"Cancellation batch size: {CANCELLATION_BATCH_SIZE}")
# print(f"Total cancellation batches: {total_cancellation_batches}")

# # Process cancellation data in batches
# successful_cancellation_batches = 0
# failed_cancellation_batches = 0

# for batch_num in range(1, total_cancellation_batches + 1):
#     start_idx = (batch_num - 1) * CANCELLATION_BATCH_SIZE
#     end_idx = min(batch_num * CANCELLATION_BATCH_SIZE, total_cancellation_rows)
    
#     batch_df = cancellation_df.iloc[start_idx:end_idx].copy()
    
#     print(f"\nProcessing cancellation batch {batch_num}/{total_cancellation_batches} (rows {start_idx + 1}-{end_idx})")
    
#     success = process_cancellation_batch(batch_df, batch_num, total_cancellation_batches)
    
#     if success:
#         successful_cancellation_batches += 1
#     else:
#         failed_cancellation_batches += 1

# print(f"\n=== Cancellation Batch Processing Complete ===")
# print(f"Successful cancellation batches: {successful_cancellation_batches}/{total_cancellation_batches}")
# print(f"Failed cancellation batches: {failed_cancellation_batches}/{total_cancellation_batches}")
# print(f"Cancellation success rate: {(successful_cancellation_batches/total_cancellation_batches)*100:.1f}%")

# # ------------------------
# # Bulk create Task with Batch Processing
# # ------------------------

# def process_batch(batch_df, batch_number, total_batches):
#     """
#     Process a single batch of data for bulk task creation with retry logic for ongoing task errors
#     """
#     import json
#     import re

#     current_batch = batch_df.copy()
#     max_retries = 10000000000  # Maximum number of retries to prevent infinite loops
#     retry_count = 0

#     while retry_count < max_retries and len(current_batch) > 0:
#         # Create a temporary file for this batch
#         batch_file_path = f"data_for_SF_batch_{batch_number}_retry_{retry_count}.xlsx"

#         try:
#             # Save batch DataFrame to an Excel file
#             current_batch.to_excel(batch_file_path, index=False)

#             # Prepare the request
#             url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/creation"

#             headers = {
#                 "accept": "application/json, text/plain, */*",
#                 "authorization": f"Bearer {AT}",
#                 "country_identifier": "EG",
#                 "language": "AR",
#                 "origin": "https://logistics.maxab.info",
#                 "referer": "https://logistics.maxab.info/",
#                 "user-agent": "Mozilla/5.0",
#             }

#             files = {
#                 "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
#             }

#             # Send POST request with file upload
#             response = requests.post(url, headers=headers, files=files)

#             print(f"Batch {batch_number}/{total_batches} (retry {retry_count}) - Response: {response.status_code}")

#             if response.status_code in [200, 201]:
#                 print(f"✓ Batch {batch_number} processed successfully after {retry_count} retries")
#                 return True
#             elif response.status_code == 400:
#                 # Try to parse the error response
#                 try:
#                     error_data = json.loads(response.text)
#                     error_message = error_data.get('message', '')

#                     # Check if it's an ongoing task error
#                     if 'RETAILER_HAS_ONGOING_TASK' in error_data.get('error_code', ''):
#                         # Extract retailer ID from the error message
#                         # Pattern: "Retailer With ID 771554 Has Ongoing Task for the same workflow"
#                         match = re.search(r'Retailer With ID (\d+) Has Ongoing Task', error_message)
#                         if match:
#                             problematic_retailer_id = int(match.group(1))
#                             print(f"⚠️  Removing retailer ID {problematic_retailer_id} with ongoing task")

#                             # Remove the problematic retailer from the batch
#                             current_batch = current_batch[current_batch['Retailer ID'] != problematic_retailer_id]

#                             if len(current_batch) == 0:
#                                 print(f"✗ Batch {batch_number} failed - no valid retailers remaining")
#                                 return False

#                             retry_count += 1
#                             print(f"🔄 Retrying batch {batch_number} with {len(current_batch)} remaining retailers")
#                             continue
#                         else:
#                             print(f"✗ Could not extract retailer ID from error message: {error_message}")
#                             return False
#                     else:
#                         # Other 400 error, not related to ongoing tasks
#                         print(f"✗ Batch {batch_number} failed with 400 error: {error_message}")
#                         return False

#                 except json.JSONDecodeError:
#                     print(f"✗ Could not parse error response: {response.text}")
#                     return False
#             else:
#                 # Other error status codes
#                 print(f"✗ Batch {batch_number} failed with status {response.status_code}")
#                 print(f"Response: {response.text}")
#                 return False

#         except Exception as e:
#             print(f"✗ Error processing batch {batch_number}: {str(e)}")
#             return False

#         finally:
#             # Clean up temporary file
#             if os.path.exists(batch_file_path):
#                 os.remove(batch_file_path)

#     if retry_count >= max_retries:
#         print(f"✗ Batch {batch_number} failed after {max_retries} retries")
#         return False

#     return True

# # Calculate batch size and number of batches
# BATCH_SIZE = 500
# total_rows = len(creation_df)
# total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division

# print(f"Total rows to process: {total_rows}")
# print(f"Batch size: {BATCH_SIZE}")
# print(f"Total batches: {total_batches}")

# # Process data in batches
# successful_batches = 0
# failed_batches = 0
# total_rows_created = 0  # Track actual rows created

# for batch_num in range(1, total_batches + 1):
#     start_idx = (batch_num - 1) * BATCH_SIZE
#     end_idx = min(batch_num * BATCH_SIZE, total_rows)

#     batch_df = creation_df.iloc[start_idx:end_idx].copy()
#     batch_row_count = len(batch_df)  # Number of rows in this batch

#     print(f"\nProcessing batch {batch_num}/{total_batches} (rows {start_idx + 1}-{end_idx})")

#     success = process_batch(batch_df, batch_num, total_batches)

#     if success:
#         successful_batches += 1
#         total_rows_created += batch_row_count  # Add successful batch rows to total
#     else:
#         failed_batches += 1

# print(f"\n=== Batch Processing Complete ===")
# print(f"Successful batches: {successful_batches}/{total_batches}")
# print(f"Failed batches: {failed_batches}/{total_batches}")
# print(f"Success rate: {(successful_batches/total_batches)*100:.1f}%")
# print(f"Total rows created: {total_rows_created}")

# # Store creation statistics
# creation_stats = {
#     'total_batches': total_batches,
#     'successful_batches': successful_batches,
#     'failed_batches': failed_batches,
#     'success_rate': (successful_batches/total_batches)*100 if total_batches > 0 else 0,
#     'total_rows_created': total_rows_created
# }



#--------------------------
# Export Data
#--------------------------

import requests

url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/export"

# Get current date in YYYY-MM-DD format
current_date = datetime.now().strftime("%Y-%m-%d")
querystring = {"date_from": current_date, "date_to": current_date, "status_ids": "1"}

payload = ""
headers = {"Authorization": f"Bearer {AT}"}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

print(f"2. Export Data Response: {response.status_code}")



response = requests.get(url, headers=headers, params=querystring)
#--------------------------
# Merge DataFrames
#--------------------------

# Load Excel directly into pandas
export_df = pd.read_excel(BytesIO(response.content))
export_df = export_df[export_df['Status'] == 'CREATED']

import pandas as pd

# Assuming your DataFrames are already loaded: export_df and df

# Merge on common columns
merged_df = export_df.merge(
    df[['RETAILER_ID', 'AGENT_ID']],   # only keep necessary columns
    how='left',                        # keep all rows from export_df
    left_on='Retailer Id',             # column name in export_df
    right_on='RETAILER_ID'             # column name in df
)

# Drop duplicate 'Retailer ID' (optional, since export_df already has 'Retailer Id')
merged_df = merged_df.drop_duplicates(subset=['RETAILER_ID', 'Task Id'])
merged_df = merged_df.drop(columns=['RETAILER_ID'])

merged_df = merged_df[['Task Id', 'AGENT_ID']]
merged_df = merged_df.drop(merged_df[merged_df['AGENT_ID'] == 10685].index)




#--------------------------
# Bulk Dispatch Tasks with Batch Processing
#--------------------------

def process_dispatch_batch(batch_df, batch_number, total_batches):
    """
    Process a single batch of data for bulk task dispatching with retry logic for agent not found errors
    """
    import json
    import re
    
    current_batch = batch_df.copy()
    max_retries = 10  # Maximum number of retries to prevent infinite loops
    retry_count = 0
    
    while retry_count < max_retries and len(current_batch) > 0:
        # Create a temporary file for this batch
        batch_file_path = f"data_for_BD_batch_{batch_number}_retry_{retry_count}.xlsx"
        
        try:
            # Save batch DataFrame to an Excel file
            current_batch.to_excel(batch_file_path, index=False)
            
            # Prepare the request
            url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/dispatching"
            
            headers = {
                "accept": "application/json, text/plain, */*",
                "authorization": f"Bearer {AT}",
                "country_identifier": "EG",
                "language": "AR",
                "origin": "https://logistics.maxab.info",
                "referer": "https://logistics.maxab.info/",
                "user-agent": "Mozilla/5.0",
            }
            
            files = {
                "file": (batch_file_path, open(batch_file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
            }
            
            # Send POST request with file upload
            response = requests.post(url, headers=headers, files=files)
            
            print(f"Dispatch Batch {batch_number}/{total_batches} (retry {retry_count}) - Response: {response.status_code}")
            
            if response.status_code in [200, 201]:
                print(f"✓ Dispatch Batch {batch_number} processed successfully after {retry_count} retries")
                return True
            elif response.status_code == 404:
                # Try to parse the error response
                try:
                    error_data = json.loads(response.text)
                    error_message = error_data.get('message', '')
                    
                    # Check if it's an agent not found error
                    if 'agent.NotFound' in error_data.get('error_code', ''):
                        # Extract agent ID and row number from the error message
                        # Pattern: "Row: 141, Agent with ID 485 is not found"
                        match = re.search(r'Row: (\d+), Agent with ID (\d+) is not found', error_message)
                        if match:
                            problematic_row = int(match.group(1))
                            problematic_agent_id = int(match.group(2))
                            print(f"⚠️  Removing agent ID {problematic_agent_id} (row {problematic_row}) - agent not found")
                            
                            # Remove the problematic agent from the batch
                            # Note: We need to remove by agent ID since that's what's causing the issue
                            current_batch = current_batch[current_batch['AGENT_ID'] != problematic_agent_id]
                            
                            if len(current_batch) == 0:
                                print(f"✗ Dispatch Batch {batch_number} failed - no valid agents remaining")
                                return False
                            
                            retry_count += 1
                            print(f"🔄 Retrying dispatch batch {batch_number} with {len(current_batch)} remaining agents")
                            continue
                        else:
                            print(f"✗ Could not extract agent ID from error message: {error_message}")
                            return False
                    else:
                        # Other 404 error, not related to agent not found
                        print(f"✗ Dispatch Batch {batch_number} failed with 404 error: {error_message}")
                        return False
                        
                except json.JSONDecodeError:
                    print(f"✗ Could not parse error response: {response.text}")
                    return False
            else:
                # Other error status codes
                print(f"✗ Dispatch Batch {batch_number} failed with status {response.status_code}")
                print(f"Response: {response.text}")
                return False
                
        except Exception as e:
            print(f"✗ Error processing dispatch batch {batch_number}: {str(e)}")
            return False
            
        finally:
            # Clean up temporary file
            if os.path.exists(batch_file_path):
                os.remove(batch_file_path)
    
    if retry_count >= max_retries:
        print(f"✗ Dispatch Batch {batch_number} failed after {max_retries} retries")
        return False
    
    return True

# Calculate batch size and number of batches for dispatch
DISPATCH_BATCH_SIZE = 500
total_dispatch_rows = len(merged_df)
total_dispatch_batches = (total_dispatch_rows + DISPATCH_BATCH_SIZE - 1) // DISPATCH_BATCH_SIZE  # Ceiling division

print(f"\n=== Bulk Dispatch Tasks ===")
print(f"Total dispatch rows to process: {total_dispatch_rows}")
print(f"Dispatch batch size: {DISPATCH_BATCH_SIZE}")
print(f"Total dispatch batches: {total_dispatch_batches}")

# Process dispatch data in batches
successful_dispatch_batches = 0
failed_dispatch_batches = 0

for batch_num in range(1, total_dispatch_batches + 1):
    start_idx = (batch_num - 1) * DISPATCH_BATCH_SIZE
    end_idx = min(batch_num * DISPATCH_BATCH_SIZE, total_dispatch_rows)
    
    batch_df = merged_df.iloc[start_idx:end_idx].copy()
    
    print(f"\nProcessing dispatch batch {batch_num}/{total_dispatch_batches} (rows {start_idx + 1}-{end_idx})")
    
    success = process_dispatch_batch(batch_df, batch_num, total_dispatch_batches)
    
    if success:
        successful_dispatch_batches += 1
    else: 
        failed_dispatch_batches += 1

print(f"\n=== Dispatch Batch Processing Complete ===")
print(f"Successful dispatch batches: {successful_dispatch_batches}/{total_dispatch_batches}")
print(f"Failed dispatch batches: {failed_dispatch_batches}/{total_dispatch_batches}")
print(f"Dispatch success rate: {(successful_dispatch_batches/total_dispatch_batches)*100:.1f}%")



/home/ec2-user/service_account_key.json
2. Export Data Response: 200


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")



=== Bulk Dispatch Tasks ===
Total dispatch rows to process: 9186
Dispatch batch size: 500
Total dispatch batches: 19

Processing dispatch batch 1/19 (rows 1-500)
Dispatch Batch 1/19 (retry 0) - Response: 200
✓ Dispatch Batch 1 processed successfully after 0 retries

Processing dispatch batch 2/19 (rows 501-1000)
Dispatch Batch 2/19 (retry 0) - Response: 200
✓ Dispatch Batch 2 processed successfully after 0 retries

Processing dispatch batch 3/19 (rows 1001-1500)
Dispatch Batch 3/19 (retry 0) - Response: 200
✓ Dispatch Batch 3 processed successfully after 0 retries

Processing dispatch batch 4/19 (rows 1501-2000)
Dispatch Batch 4/19 (retry 0) - Response: 200
✓ Dispatch Batch 4 processed successfully after 0 retries

Processing dispatch batch 5/19 (rows 2001-2500)
Dispatch Batch 5/19 (retry 0) - Response: 200
✓ Dispatch Batch 5 processed successfully after 0 retries

Processing dispatch batch 6/19 (rows 2501-3000)
Dispatch Batch 6/19 (retry 0) - Response: 200
✓ Dispatch Batch 6 processe

In [9]:
creation_df

,RETAILER_ID,Workflow ID,Date,Description
0,534507,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
1,534507,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
2,388287,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
3,388287,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
4,392449,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
...,...,...,...,...
6113,401871,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
6114,401169,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
6115,401169,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.
6116,405983,1,2025-08-27,اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.


In [12]:
# Find duplicate retailer_id rows
duplicates = creation_df[creation_df.duplicated(subset=['RETAILER_ID'], keep=False)]

# Show them
print(duplicates)

# Count how many duplicates each retailer_id has
# dup_counts = creation_df['retailer_id'].value_counts()
# dup_counts = dup_counts[dup_counts > 1]

# print(dup_counts)


      RETAILER_ID  Workflow ID        Date  \
0          534507            1  2025-08-27   
1          534507            1  2025-08-27   
2          388287            1  2025-08-27   
3          388287            1  2025-08-27   
4          392449            1  2025-08-27   
...           ...          ...         ...   
6113       401871            1  2025-08-27   
6114       401169            1  2025-08-27   
6115       401169            1  2025-08-27   
6116       405983            1  2025-08-27   
6117       405983            1  2025-08-27   

                               Description  
0     اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.  
1     اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.  
2     اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.  
3     اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.  
4     اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.  
...                                    ...  
6113  اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.  
6114  اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.  
6115  اعمل طلب بـ3500 من 'بضاعة مكسب' فقط.

In [3]:
df = ret_metabase("Egypt", 62832)

/home/ec2-user/service_account_key.json


In [4]:
df

,Task ID
0,6458627
1,6458020
2,6457980
3,6458028
4,6458030
...,...
2317,6455524
2318,6454516
2319,6455140
2320,6453850


In [67]:
AT = 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4cnE5b1ZqaXRFdjdNeExyTWpzS3NkdUl3eWdxZnZlVkM4T0Vfb3dvY3N3In0.eyJleHAiOjE3NTU1MTEzMjcsImlhdCI6MTc1NTUxMTAyNywiYXV0aF90aW1lIjoxNzU1NTAwMzY5LCJqdGkiOiIyMDcyM2RiNi02YWNiLTQzYzAtYjUzNi03YmI3MjQxMGY5ODYiLCJpc3MiOiJodHRwczovL3Nzby5tYXhhYi5pbmZvL2F1dGgvcmVhbG1zL21heGFiIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjYwZDc3NzAxLTZiYzctNDE0Mi05YmZjLTMxZmMyYjY1MjJhNCIsInR5cCI6IkJlYXJlciIsImF6cCI6Indvcmtmb3JjZS1wb3J0YWwiLCJub25jZSI6IjhhYmNlMjBiLTUzZjMtNDJkYi04ZjI0LTM2ODM2ZjM3NWZmNyIsInNlc3Npb25fc3RhdGUiOiIzZWIzNzNkMi04NDNjLTQ1OWUtYjI3OS0wOWExYjYxNjU3ZTEiLCJhY3IiOiIwIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9naXN0aWNzLm1heGFiLmluZm8iLCJodHRwczovL3dvcmtmb3JjZS5tYXhhYi5pbmZvIiwiaHR0cHM6Ly9vbXMubWF4YWIuaW5mbyJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJzaWQiOiIzZWIzNzNkMi04NDNjLTQ1OWUtYjI3OS0wOWExYjYxNjU3ZTEiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwibmFtZSI6IkJhc3NlbSBNYW1kb3VoIiwicHJlZmVycmVkX3VzZXJuYW1lIjoiYmFzc2VtLm1hbWRvdWhAbWF4YWIuaW8iLCJnaXZlbl9uYW1lIjoiQmFzc2VtIiwiZmFtaWx5X25hbWUiOiJNYW1kb3VoIiwiZW1haWwiOiJiYXNzZW0ubWFtZG91aEBtYXhhYi5pbyJ9.dF07AlHlDMok3uPODn-cBJUDA_rwkcYBFLEp9K-pbfWjQ13cmDBL2wH1cJYHw8nnLt5nPFgnAxQ7wQipWHGOR6rBz0HVlKoxVdFXw-iq-1NrzU3O6nhN_xBOCErj_1J3BWHxy8-Qki6g6fRgtiWipqB2jfF3fgG1P1Wvn7siYX0Loa-aqSmJLaSHUKW9IxR1Pt6etShjXzZB1rUwRBiFcT-DmuIi7JjuVpsqUMR4uuUpRM2VFo8UM01EklQI_ZsD3R45LXwqZcfasPenHoQIFPb9fPCwj0AC5t34SINqNgtiQtTyt8bEo57zxSExYYSEngHMGsM6PFWBvp-i1hJ0kw'

In [7]:

df = ret_metabase("Egypt", 62832)
AT = get_jwt()

file_path = "data_for_SF.xlsx"
df.to_excel(file_path, index=False)

# Step 2: Prepare the request
url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/cancellation"

headers = {
    "accept": "application/json, text/plain, */*",
    "authorization": f"Bearer {AT}",
    "country_identifier": "EG",
    "language": "AR",
    "origin": "https://logistics.maxab.info",
    "referer": "https://logistics.maxab.info/",
    "user-agent": "Mozilla/5.0",
}

files = {
    "file": (file_path, open(file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
}

# Step 3: Send POST request with file upload
response = requests.post(url, headers=headers, files=files)

print(response.status_code)
print(response.text)


200



In [28]:
print(response.text)

In [6]:
import requests

url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/export"

querystring = {"date_from":"2025-08-18","date_to":"2025-08-18","status_ids":"1"}

payload = ""
headers = {"Authorization": f"Bearer {AT}"}

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

print(response)

<Response [200]>


In [76]:
response = requests.get(url, headers=headers, params=querystring)

# Load Excel directly into pandas
export_df = pd.read_excel(BytesIO(response.content))

# Show preview
export_df

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Task Id,Workflow Name,Retailer Name,Retailer Id,Status,Date,Agent Name,Visit Id
0,6412928,الاحتفاظ بالعميل,mohamed,917698,FAILED,2025-08-18,Ahmed Musa Rezk Mohamed,5582128.0
1,6412929,الاحتفاظ بالعميل,العمدة,866857,DISPATCHED,2025-08-18,Abdelhamed Abdelkadr Abdelhamd Abdelhamed,5582761.0
2,6412930,الاحتفاظ بالعميل,يوسف,739018,DISPATCHED,2025-08-18,Abdelhamed Abdelkadr Abdelhamd Abdelhamed,5582762.0
3,6412931,الاحتفاظ بالعميل,جمال,254284,DISPATCHED,2025-08-18,Abdelhamed Abdelkadr Abdelhamd Abdelhamed,5582763.0
4,6412932,الاحتفاظ بالعميل,احمد,748675,DISPATCHED,2025-08-18,Abdelhamed Abdelkadr Abdelhamd Abdelhamed,5582764.0
...,...,...,...,...,...,...,...,...
4830,6417758,الاحتفاظ بالعميل,محمود,23157,CREATED,2025-08-18,NaN,NaN
4831,6417759,الاحتفاظ بالعميل,محمد عطا,36945,CREATED,2025-08-18,NaN,NaN
4832,6417760,الاحتفاظ بالعميل,Homam ALRAWAS,27356,CREATED,2025-08-18,NaN,NaN
4833,6417761,الاحتفاظ بالعميل,مصري محمد احمد,811665,CREATED,2025-08-18,NaN,NaN


In [77]:
export_df = export_df[export_df['Status'] == 'CREATED']

In [78]:
export_df

,Task Id,Workflow Name,Retailer Name,Retailer Id,Status,Date,Agent Name,Visit Id
4815,6417743,الاحتفاظ بالعميل,شنوده,907055,CREATED,2025-08-18,NaN,NaN
4816,6417744,الاحتفاظ بالعميل,مصطفي الشيخ,76186,CREATED,2025-08-18,NaN,NaN
4817,6417745,الاحتفاظ بالعميل,عماد,87516,CREATED,2025-08-18,NaN,NaN
4818,6417746,الاحتفاظ بالعميل,عبود ايوب,80049,CREATED,2025-08-18,NaN,NaN
4819,6417747,الاحتفاظ بالعميل,فتحي,697454,CREATED,2025-08-18,NaN,NaN
4820,6417748,الاحتفاظ بالعميل,كيرلس سامي,698278,CREATED,2025-08-18,NaN,NaN
4821,6417749,الاحتفاظ بالعميل,عبدو,755172,CREATED,2025-08-18,NaN,NaN
4822,6417750,الاحتفاظ بالعميل,احمد,99486,CREATED,2025-08-18,NaN,NaN
4823,6417751,الاحتفاظ بالعميل,علي,495500,CREATED,2025-08-18,NaN,NaN
4824,6417752,الاحتفاظ بالعميل,الحاج محمود,27611,CREATED,2025-08-18,NaN,NaN


In [79]:
import pandas as pd

# Assuming your DataFrames are already loaded: export_df and df

# Merge on common columns
merged_df = export_df.merge(
    df[['Retailer ID', 'AGENT_ID']],   # only keep necessary columns
    how='left',                        # keep all rows from export_df
    left_on='Retailer Id',             # column name in export_df
    right_on='Retailer ID'             # column name in df
)

# Drop duplicate 'Retailer ID' (optional, since export_df already has 'Retailer Id')
merged_df = merged_df.drop(columns=['Retailer ID'])

print(merged_df.head())


   Task Id     Workflow Name Retailer Name  Retailer Id   Status        Date  \
0  6417743  الاحتفاظ بالعميل        شنوده        907055  CREATED  2025-08-18   
1  6417744  الاحتفاظ بالعميل   مصطفي الشيخ        76186  CREATED  2025-08-18   
2  6417745  الاحتفاظ بالعميل          عماد        87516  CREATED  2025-08-18   
3  6417746  الاحتفاظ بالعميل     عبود ايوب        80049  CREATED  2025-08-18   
4  6417747  الاحتفاظ بالعميل          فتحي       697454  CREATED  2025-08-18   

  Agent Name  Visit Id  AGENT_ID  
0        NaN       NaN      1004  
1        NaN       NaN      1004  
2        NaN       NaN      1004  
3        NaN       NaN      1004  
4        NaN       NaN      1004  


In [80]:
# merged_df['Agent Name'] = 'Bassem'
# merged_df['Visit Id'] = '1010'
merged_df

,Task Id,Workflow Name,Retailer Name,Retailer Id,Status,Date,Agent Name,Visit Id,AGENT_ID
0,6417743,الاحتفاظ بالعميل,شنوده,907055,CREATED,2025-08-18,NaN,NaN,1004
1,6417744,الاحتفاظ بالعميل,مصطفي الشيخ,76186,CREATED,2025-08-18,NaN,NaN,1004
2,6417745,الاحتفاظ بالعميل,عماد,87516,CREATED,2025-08-18,NaN,NaN,1004
3,6417746,الاحتفاظ بالعميل,عبود ايوب,80049,CREATED,2025-08-18,NaN,NaN,1004
4,6417747,الاحتفاظ بالعميل,فتحي,697454,CREATED,2025-08-18,NaN,NaN,1004
5,6417748,الاحتفاظ بالعميل,كيرلس سامي,698278,CREATED,2025-08-18,NaN,NaN,1004
6,6417749,الاحتفاظ بالعميل,عبدو,755172,CREATED,2025-08-18,NaN,NaN,1004
7,6417750,الاحتفاظ بالعميل,احمد,99486,CREATED,2025-08-18,NaN,NaN,1004
8,6417751,الاحتفاظ بالعميل,علي,495500,CREATED,2025-08-18,NaN,NaN,1004
9,6417752,الاحتفاظ بالعميل,الحاج محمود,27611,CREATED,2025-08-18,NaN,NaN,1004


In [81]:
merged_df = merged_df[['Task Id', 'AGENT_ID']]
merged_df

,Task Id,AGENT_ID
0,6417743,1004
1,6417744,1004
2,6417745,1004
3,6417746,1004
4,6417747,1004
5,6417748,1004
6,6417749,1004
7,6417750,1004
8,6417751,1004
9,6417752,1004


In [82]:
import pandas as pd
import requests

# Example: assume you already have your DataFrame
# data_for_SF = pd.DataFrame(...)

# Step 1: Save DataFrame to an Excel file
file_path = "data_for_BD.xlsx"
merged_df.to_excel(file_path, index=False)

# Step 2: Prepare the request
url = "https://api.maxab.info/logistics/task-based/api/portal/v1/tasks/sheets/dispatching"

headers = {
    "accept": "application/json, text/plain, */*",
    "authorization": f"Bearer {AT}",
    "country_identifier": "EG",
    "language": "AR",
    "origin": "https://logistics.maxab.info",
    "referer": "https://logistics.maxab.info/",
    "user-agent": "Mozilla/5.0",
}

files = {
    "file": (file_path, open(file_path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
}

# Step 3: Send POST request with file upload
response = requests.post(url, headers=headers, files=files)

print(response.status_code)
print(response.text)


200



In [45]:
import requests
AT = get_jwt()
url = "https://api.maxabke.com/logistics/task-based/api/portal/v1/tasks/sheets/creation"

payload = {}
files=[
  ('file',('bulk-create-tasks.xls',open(<PATH_TO_EXCEL_FILE>','rb'),'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
]

headers = {
  'Authorization': f'Bearer {AT}'
}

response = requests.request("POST", url, headers=headers, data=payload, files=files)
response.status_code

401

In [53]:
print("Status:", response.status_code)
print("Headers:", response.headers)
try:
    print("JSON:", response.json())
except ValueError:
    print("Body:", response.text)


Status: 401
Headers: {'Date': 'Wed, 13 Aug 2025 09:03:54 GMT', 'Content-Length': '0', 'Connection': 'keep-alive', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'WWW-Authenticate': 'Bearer realm="MaxABKenya", error="invalid_token", error_description="Didn\'t find publicKey for specified kid", Bearer realm="MaxABKenya", error="invalid_token", error_description="Didn\'t find publicKey for specified kid"', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY'}
Body: 


In [8]:
AT = get_jwt()
AT

{'error': 'invalid_grant', 'error_description': 'Invalid user credentials'}

In [4]:
fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
fintech_service_account

{'email_name': 'fintech-lambda@maxab.io',
 'email_password': 'q0Rkh%ONKf9@bznHU#',
 'personal_email': 'mahmoud.alheeny@maxab.io',
 'personal_password': 'MMGAFintech123!'}

In [7]:
import requests
from common_functions import get_secret
import json
def get_jwt():
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
    

    r = requests.post("https://sso.maxabsa.com/auth/realms/MaxABKenya/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
    )
    fullJwtResponse = r.json()
    # jwt_access_token = fullJwtResponse['access_token']
        

        
        
    
    return fullJwtResponse

In [9]:
AT = get_jwt()
AT

{'error': 'invalid_grant', 'error_description': 'Invalid user credentials'}

In [4]:
import requests
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
    
    if country == 'KE': ## Kenya 
        r = requests.post("https://sso.maxabsa.com/auth/realms/MaxABKenya/protocol/openid-connect/token",
            headers={"Content-Type":"application/x-www-form-urlencoded"},
            data=payload
        )
        fullJwtResponse = r.json()
        jwt_access_token = fullJwtResponse['access_token']
        
    elif country == 'MA': ## Morocco
        r = requests.post("https://sso.maxabma.com/auth/realms/maxab/protocol/openid-connect/token",
            headers={"Content-Type":"application/x-www-form-urlencoded"},
            data=payload
        )

        fullJwtResponse = r.json()
        jwt_access_token = fullJwtResponse['access_token']
        
    elif country == 'ZA': ## Zambia
        r = requests.post("https://sso.maxabsa.com/auth/realms/maxab/protocol/openid-connect/token",
            headers={"Content-Type":"application/x-www-form-urlencoded"},
            data=payload
        )

        fullJwtResponse = r.json()
        jwt_access_token = fullJwtResponse['access_token']
            
    else: ## Egypt 
        r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
            headers={"Content-Type":"application/x-www-form-urlencoded"},
            data=payload
            )

        fullJwtResponse = r.json()
        jwt_access_token = fullJwtResponse['access_token']
        
        
    
    return jwt_access_token

In [ ]:
import requests

# Step 1: Get your data
df = ret_metabse("EGYPT", 1111)  

# Step 2: Save DataFrame to Excel
file_path = "/tmp/bulk-dispatch-tasks.xlsx"
df.to_excel(file_path, index=False)

# Step 3: Prepare request
url = "https://api.maxabke.com/logistics/task-based/api/portal/v1/tasks/sheets/dispatching"
headers = {
    "Authorization": f"Bearer {get_jwt('EG')}"  # using your get_jwt() function
}
files = [
    ('file', ('bulk-dispatch-tasks.xlsx', open(file_path, 'rb'), 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'))
]

# Step 4: Send POST request
response = requests.post(url, headers=headers, files=files)
print(response.status_code, response.text)
